In [1]:
import requests
from bs4 import BeautifulSoup
import serpscrap,pandas as pd
from lxml.html import fromstring
import numpy as np

In [2]:
df = pd.read_csv('sample_2000_M5.csv')
df = df[df['WebsiteAddressTxt'] != 'NONE']
df.head(15)

,ActivityOrMissionDesc,BooksInCareOfDetail/USAddress/ZIPCode,CYSalariesCompEmpBnftPaidAmt,CYTotalExpensesAmt,CYTotalProfFndrsngExpnsAmt,CYTotalRevenueAmt,Desc,EIN,EmployeeCnt,FormType,...,TaxPeriod,TotalAssetsGrp/EOYAmt,TotalContributionsAmt,TotalEmployeeCnt,TotalLiabilitiesGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt,TotalVolunteersCnt,URL,WebsiteAddressTxt,_id
0,WE DISCOVER PRECISE GENOMIC SOLUTIONS FOR DISE...,4609.0,150912017,281509812,175252,304318753,DEVELOP AND PROVIDE GENETIC RESOURCES - PLEASE...,10211513,2080,990,...,201512,754989281,89944366.0,2080,168362562,586626719,30.0,https://s3.amazonaws.com/irs-form-990/20162320...,WWW.JAX.ORG,010211513_201512
1,"TO RAISE, MAINTAIN AND HOLD FUNDS FOR THE BENE...",33612.0,2682097,19796283,999310,16803617,H. LEE MOFFITT CANCER CENTER & RESEARCH INSTIT...,593238636,34,990,...,201606,115979973,11856576.0,34,4565154,111414819,31.0,https://s3.amazonaws.com/irs-form-990/20162319...,WWW.MOFFITT.ORG/GIVE-BACK,593238636_201606
2,TEEN CHALLENGE IS COMMITTED TO ENABLILNG AND E...,92507.0,4826398,10164136,0,10879246,RESIDENTIAL TREATMENT TEEN CHALLENGE OPERATES ...,952683852,133,990,...,201603,13965228,6542879.0,133,2286082,11679146,650.0,https://s3.amazonaws.com/irs-form-990/20161315...,WWW.TEENCHALLENGE.ORG,952683852_201603
3,"EDUCATION, DISPLAY AND UPKEEP OF MAJOR AUTOMOT...",2445.0,326069,700536,0,753031,THE LARZ ANDERSON AUTO MUSEUM SUPPORT THE COLL...,42117470,12,990,...,201512,487237,264842.0,12,75898,411339,15.0,https://s3.amazonaws.com/irs-form-990/20163316...,WWW.LARZANDERSON.ORG,042117470_201512
4,THEATRICAL PRODUCTIONS,28563.0,51158,212711,0,289496,COMMUNITY THEATRE PRODUCTIONS,581409097,2,990,...,201512,307538,102894.0,2,48786,258752,250.0,https://s3.amazonaws.com/irs-form-990/20161319...,WWW.NEWBERNCIVICTHEATRE.ORG,581409097_201512
5,"TO STUDY, PRESERVE AND CELEBRATE THE CULTURE A...",6426.0,447235,1052697,0,1558175,"THE ORGANIZATION COLLECTS, PRESERVES AND STUDI...",237417579,17,990,...,201606,5392670,1178948.0,17,135177,5257493,NaN,https://s3.amazonaws.com/irs-form-990/20163320...,WWW.CTRIVERMUSEUM.ORG,237417579_201606
6,TO PROVIDE THEOLOGICAL EDUCATION SERVICES,90710.0,37905,194384,0,158677,PROVIDE THEOLOGICAL EDUCATION SERVICE,731679393,3,990,...,201512,25504,73954.0,3,2680,22824,NaN,https://s3.amazonaws.com/irs-form-990/20161320...,WWW.MYCCU.US,731679393_201512
7,TO PROVIDE HIGH QUALITY HOME HEALTH CARE AND H...,91711.0,28216635,38763703,0,38658103,THE HOSPICE SERVICES PROGRAM HELPS PROVIDE COM...,951733155,600,990,...,201512,26998880,146267.0,600,13786894,13211986,165.0,https://s3.amazonaws.com/irs-form-990/20160307...,WWW.VNASOCAL.ORG,951733155_201512
8,THE MISSION OF THE GREATER HALL CHAMBER OF COM...,30501.0,893732,1550194,0,1443975,GENERAL CHAMBER ACTIVITIES: MEMBERSHIP SERVICE...,580251406,17,990,...,201606,5402808,795627.0,17,405482,4997326,1000.0,https://s3.amazonaws.com/irs-form-990/20162320...,WWW.GHCC.COM,580251406_201606
9,TO SERVE AS A CATALYST FOR COLLABORATIVE ACTIV...,29601.0,114156,846186,0,1734018,CANCER SURVIVORS PARK - ENGAGES COMMUNITY STAK...,571085380,2,990,...,201606,4628157,1613026.0,2,7098,4621059,NaN,https://s3.amazonaws.com/irs-form-990/20161320...,CANCERSURVIVORSPARK.ORG,571085380_201606


# Get Proxies

In [3]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

proxies = get_proxies()
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
print(proxies)

proxies = np.random.choice(list(proxies), len(df))
df['Proxy'] =  proxies



{'190.249.157.89:56294'}


In [4]:
def getSnippets(ws,proxy):
    
    try:

        query = ws.replace(" ","")
        query = "https://www.bing.com/search?q=" + query
        print(query)
        r = requests.get(query,proxies={"http": proxy})
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')

        for s in soup.find_all("li",{"class" : "b_algo"}):
            snip = s.text
            break

        return snip


    except:
        return None

In [ ]:
import multiprocessing
with multiprocessing.Pool() as p:
    df['WebSnippet'] = df.apply(lambda x: getSnippets(x['WebsiteAddressTxt'], x['Proxy']),axis=1)
    #df['WebSnippet'] = df.apply(lambda x: getSnippets(x['WebsiteAddressTxt'], x['Proxy']) if(np.all(pd.isna(x['WebSnippet4']))) else x['WebSnippet4'], axis=1)


https://www.bing.com/search?q=WWW.JAX.ORG
https://www.bing.com/search?q=WWW.MOFFITT.ORG/GIVE-BACK
https://www.bing.com/search?q=WWW.TEENCHALLENGE.ORG
https://www.bing.com/search?q=WWW.LARZANDERSON.ORG
https://www.bing.com/search?q=WWW.NEWBERNCIVICTHEATRE.ORG
https://www.bing.com/search?q=WWW.CTRIVERMUSEUM.ORG
https://www.bing.com/search?q=WWW.MYCCU.US
https://www.bing.com/search?q=WWW.VNASOCAL.ORG
https://www.bing.com/search?q=WWW.GHCC.COM
https://www.bing.com/search?q=CANCERSURVIVORSPARK.ORG
https://www.bing.com/search?q=WWW.CHATHAMORPHEUM.ORG
https://www.bing.com/search?q=brazosvalleymuseum.org
https://www.bing.com/search?q=WILLBRIDGEOFSANTABARBARAINC.ORG
https://www.bing.com/search?q=WWW.WESTVIEWONTHEJAMES.ORG


In [10]:
len(df[df.WebSnippet.isnull()])/len(df)

0.3612644254892122

In [13]:
df = df.drop(columns=['Unnamed: 0','Proxy'])

In [78]:
outputPath = "sample_2000_M5_socialv2.csv"
df.to_csv(outputPath)